In [ ]:
from dataclasses import dataclass
from enum import StrEnum
from datetime import datetime, timedelta
import polars as pl
from collections import defaultdict, Counter
from typing import Tuple
from common.constants.column_types import (
    CPZP_SCHEMA,
    OZP_SCHEMA,
    POHLAVI_CPZP,
    TYP_UDALOSTI,
)
from common.constants.column_names import SHARED_COLUMNS, OZP_COLUMNS, CPZP_COLUMNS
import pickle
from common.constants.objects import (
    Person,
    Gender,
    AgeCohort,
    Prescription,
    PrescriptionType,
)
from common.utils import draw_chart
import matplotlib.pyplot as plt
import numpy as np
import os

pl.Config.set_tbl_rows(20)
pl.Config.set_tbl_cols(60)

POJISTOVNA = "cpzp"


with open(f"./DATACON_data/{POJISTOVNA}_persons.pkl", "rb") as f:
    persons: list[Person] = pickle.load(f)

In [9]:
vax_dates_map: dict[AgeCohort, dict[int, list[datetime]]] = defaultdict(dict)

for person in persons:
    if person.died_at or not person.vaccines:
        continue
    for vax in person.vaccines:
        if vax.dose_number not in vax_dates_map[vax.age_cohort]:
            vax_dates_map[vax.age_cohort][vax.dose_number] = []
        vax_dates_map[vax.age_cohort][vax.dose_number].append(vax.date)

max_vax_dates_map: dict[AgeCohort, dict[int, datetime]] = defaultdict(dict)

for age_cohort, doses_map in vax_dates_map.items():
    for dose_number, dates in doses_map.items():
        if not dates:
            continue

        max_date, _ = max(Counter(dates).items(), key=lambda x: x[1])
        max_vax_dates_map[age_cohort][dose_number] = max_date

print("Max vaccination dates by cohort:")
print(max_vax_dates_map)

Max vaccination dates by cohort:
defaultdict(<class 'dict'>, {<AgeCohort.BETWEEN_50_AND_70: 'between_50_and_70'>: {1: datetime.date(2021, 5, 10), 2: datetime.date(2021, 6, 21), 3: datetime.date(2021, 12, 17), 4: datetime.date(2022, 10, 21), 5: datetime.date(2023, 10, 25), 6: datetime.date(2023, 10, 11), 7: datetime.date(2023, 11, 29)}, <AgeCohort.BETWEEN_12_AND_30: 'between_12_and_30'>: {1: datetime.date(2021, 6, 9), 2: datetime.date(2021, 8, 9), 3: datetime.date(2022, 1, 28), 4: datetime.date(2022, 10, 7), 5: datetime.date(2023, 11, 2), 6: datetime.date(2023, 10, 25)}, <AgeCohort.BETWEEN_30_AND_50: 'between_30_and_50'>: {1: datetime.date(2021, 5, 19), 2: datetime.date(2021, 7, 2), 3: datetime.date(2022, 1, 7), 4: datetime.date(2022, 10, 14), 5: datetime.date(2023, 10, 17), 6: datetime.date(2023, 10, 9), 7: datetime.date(2023, 11, 28)}, <AgeCohort.MORE_THAN_70: 'more_than_70'>: {1: datetime.date(2021, 3, 11), 2: datetime.date(2021, 4, 8), 3: datetime.date(2021, 12, 2), 4: datetime.date

In [10]:
# print("\nGenerating charts for each cohort and dose...")


# for age_cohort in vax_dates_map:
#     for dose_number, dates in vax_dates_map[age_cohort].items():
#         if not dates:
#             continue

#         days_to_counts = {}
#         for date, count in Counter(dates).items():
#             days_to_counts[date] = count

#         draw_chart(
#             mapp=days_to_counts,
#             x_label="Days since first vaccination in this dose",
#             y_label="Number of vaccinations",
#             title=f"Vaccination Timeline - {age_cohort.value} - Dose {dose_number}",
#             save_location=f"out/{POJISTOVNA}/vax_period/{age_cohort.value}_dose_{dose_number}.png",
#         )

In [11]:
VAX_PERIOD_IN_DAYS = 30
PRESCRIPTION_PERIOD_IN_DAYS = 364 / 2

predpisy_map: dict[AgeCohort, dict[int, dict[datetime, int]]] = defaultdict(
    lambda: defaultdict(dict)
)

prvopredpisy_map: dict[AgeCohort, dict[int, dict[datetime, int]]] = defaultdict(
    lambda: defaultdict(dict)
)

prednison_equivs_map: dict[AgeCohort, dict[int, dict[datetime, int]]] = defaultdict(
    lambda: defaultdict(dict)
)

for p in persons:
    if p.died_at or not p.vaccines:
        continue

    for vax in p.vaccines:
        max_int_date = max_vax_dates_map[vax.age_cohort][vax.dose_number]
        if abs((vax.date - max_int_date).days) > VAX_PERIOD_IN_DAYS:
            continue

        if not p.prescriptions:
            continue

        # === PRESCRIPTIONS ===
        last_prescription_date: datetime = datetime.min.date()
        for prescription in p.prescriptions:
            if (
                abs((prescription.date - max_int_date).days)
                > PRESCRIPTION_PERIOD_IN_DAYS
            ):
                continue

            if (
                prescription.lekova_forma == "Injekční suspenze"
                or prescription.lekova_forma == "Injekční/infuzní roztok"
            ):
                if abs((last_prescription_date - prescription.date).days) < 14:
                    continue

            # === PRESCRIPTIONS COUNT ===
            last_prescription_date = prescription.date
            if prescription.date not in predpisy_map[vax.age_cohort][vax.dose_number]:
                predpisy_map[vax.age_cohort][vax.dose_number][prescription.date] = 0
            predpisy_map[vax.age_cohort][vax.dose_number][prescription.date] += 1

            # === PREDNISON EQUIV ===
            if prescription.prednison_equiv is not None:
                if (
                    prescription.date
                    not in prednison_equivs_map[vax.age_cohort][vax.dose_number]
                ):
                    prednison_equivs_map[vax.age_cohort][vax.dose_number][
                        prescription.date
                    ] = 0
                prednison_equivs_map[vax.age_cohort][vax.dose_number][
                    prescription.date
                ] += prescription.prednison_equiv

        # === FIRST PRESCRIPTIONS ===
        first_prescription = min(p.prescriptions, key=lambda x: x.date)
        if (
            abs((first_prescription.date - max_int_date).days)
            > PRESCRIPTION_PERIOD_IN_DAYS
        ):
            continue
        if (
            first_prescription.date
            not in prvopredpisy_map[vax.age_cohort][vax.dose_number]
        ):
            prvopredpisy_map[vax.age_cohort][vax.dose_number][
                first_prescription.date
            ] = 0
        prvopredpisy_map[vax.age_cohort][vax.dose_number][first_prescription.date] += 1

print(prvopredpisy_map)

defaultdict(<function <lambda> at 0x7f5870982c00>, {<AgeCohort.BETWEEN_30_AND_50: 'between_30_and_50'>: defaultdict(<class 'dict'>, {1: {datetime.date(2021, 8, 4): 17, datetime.date(2021, 9, 1): 28, datetime.date(2021, 6, 29): 18, datetime.date(2021, 2, 16): 30, datetime.date(2021, 4, 28): 23, datetime.date(2021, 3, 9): 24, datetime.date(2021, 2, 25): 22, datetime.date(2021, 10, 17): 2, datetime.date(2021, 9, 17): 19, datetime.date(2020, 11, 25): 17, datetime.date(2021, 7, 16): 13, datetime.date(2021, 8, 25): 17, datetime.date(2021, 6, 8): 24, datetime.date(2021, 8, 2): 19, datetime.date(2021, 7, 27): 20, datetime.date(2021, 8, 18): 16, datetime.date(2021, 5, 10): 25, datetime.date(2021, 6, 15): 25, datetime.date(2020, 12, 8): 22, datetime.date(2020, 12, 3): 19, datetime.date(2020, 11, 26): 16, datetime.date(2021, 9, 21): 18, datetime.date(2021, 11, 1): 33, datetime.date(2021, 8, 5): 17, datetime.date(2021, 4, 16): 19, datetime.date(2021, 1, 27): 21, datetime.date(2021, 2, 17): 23, dat

In [12]:
for age_cohort, doses_map in predpisy_map.items():
    for dose_number, dates in doses_map.items():
        if not dates:
            continue

        draw_chart(
            mapp=dates,
            x_label="Days around max intensity",
            y_label="Number of prescriptions",
            title=f"Prescriptions around max intensity - {age_cohort.value} - Dose {dose_number}",
            save_location=f"out/{POJISTOVNA}/predpisy/{age_cohort.value}_dose_{dose_number}.png",
            vertical_line=max_vax_dates_map[age_cohort][dose_number],
        )

✓ Chart saved: out/cpzp/predpisy/between_30_and_50_dose_1.png
✓ Chart saved: out/cpzp/predpisy/between_30_and_50_dose_2.png
✓ Chart saved: out/cpzp/predpisy/between_30_and_50_dose_3.png
✓ Chart saved: out/cpzp/predpisy/between_30_and_50_dose_4.png
✓ Chart saved: out/cpzp/predpisy/between_30_and_50_dose_5.png
✓ Chart saved: out/cpzp/predpisy/more_than_70_dose_1.png
✓ Chart saved: out/cpzp/predpisy/more_than_70_dose_2.png
✓ Chart saved: out/cpzp/predpisy/more_than_70_dose_3.png
✓ Chart saved: out/cpzp/predpisy/more_than_70_dose_4.png
✓ Chart saved: out/cpzp/predpisy/more_than_70_dose_5.png
✓ Chart saved: out/cpzp/predpisy/between_50_and_70_dose_1.png
✓ Chart saved: out/cpzp/predpisy/between_50_and_70_dose_2.png
✓ Chart saved: out/cpzp/predpisy/between_50_and_70_dose_3.png
✓ Chart saved: out/cpzp/predpisy/between_50_and_70_dose_4.png
✓ Chart saved: out/cpzp/predpisy/between_50_and_70_dose_5.png
✓ Chart saved: out/cpzp/predpisy/between_12_and_30_dose_2.png
✓ Chart saved: out/cpzp/predpisy/

In [13]:
for age_cohort, doses_map in prvopredpisy_map.items():
    for dose_number, dates in doses_map.items():
        if not dates:
            continue

        draw_chart(
            mapp=dates,
            x_label="Days around max intensity",
            y_label="Number of first prescriptions",
            title=f"First prescriptions around max intensity - {age_cohort.value} - Dose {dose_number}",
            save_location=f"out/{POJISTOVNA}/prvopredpisy/{age_cohort.value}_dose_{dose_number}.png",
            vertical_line=max_vax_dates_map[age_cohort][dose_number],
        )

✓ Chart saved: out/cpzp/prvopredpisy/between_30_and_50_dose_1.png
✓ Chart saved: out/cpzp/prvopredpisy/between_30_and_50_dose_2.png
✓ Chart saved: out/cpzp/prvopredpisy/between_30_and_50_dose_3.png
✓ Chart saved: out/cpzp/prvopredpisy/more_than_70_dose_1.png
✓ Chart saved: out/cpzp/prvopredpisy/more_than_70_dose_2.png
✓ Chart saved: out/cpzp/prvopredpisy/more_than_70_dose_3.png
✓ Chart saved: out/cpzp/prvopredpisy/more_than_70_dose_4.png
✓ Chart saved: out/cpzp/prvopredpisy/more_than_70_dose_5.png
✓ Chart saved: out/cpzp/prvopredpisy/between_12_and_30_dose_2.png
✓ Chart saved: out/cpzp/prvopredpisy/between_12_and_30_dose_1.png
✓ Chart saved: out/cpzp/prvopredpisy/between_12_and_30_dose_3.png
✓ Chart saved: out/cpzp/prvopredpisy/between_50_and_70_dose_1.png
✓ Chart saved: out/cpzp/prvopredpisy/between_50_and_70_dose_2.png
✓ Chart saved: out/cpzp/prvopredpisy/between_50_and_70_dose_3.png
✓ Chart saved: out/cpzp/prvopredpisy/between_50_and_70_dose_4.png


In [14]:
for age_cohort, doses_map in prednison_equivs_map.items():
    for dose_number, dates in doses_map.items():
        if not dates:
            continue

        draw_chart(
            mapp=dates,
            x_label="Days around max intensity",
            y_label="Number of prednison equivalents",
            title=f"Prednison equivalents around max intensity - {age_cohort.value} - Dose {dose_number}",
            save_location=f"out/{POJISTOVNA}/prednison_equivs/{age_cohort.value}_dose_{dose_number}.png",
            vertical_line=max_vax_dates_map[age_cohort][dose_number],
        )

✓ Chart saved: out/cpzp/prednison_equivs/between_30_and_50_dose_1.png
✓ Chart saved: out/cpzp/prednison_equivs/between_30_and_50_dose_2.png
✓ Chart saved: out/cpzp/prednison_equivs/between_30_and_50_dose_3.png
✓ Chart saved: out/cpzp/prednison_equivs/between_30_and_50_dose_4.png
✓ Chart saved: out/cpzp/prednison_equivs/between_30_and_50_dose_5.png
✓ Chart saved: out/cpzp/prednison_equivs/more_than_70_dose_1.png
✓ Chart saved: out/cpzp/prednison_equivs/more_than_70_dose_2.png
✓ Chart saved: out/cpzp/prednison_equivs/more_than_70_dose_3.png
✓ Chart saved: out/cpzp/prednison_equivs/more_than_70_dose_4.png
✓ Chart saved: out/cpzp/prednison_equivs/more_than_70_dose_5.png
✓ Chart saved: out/cpzp/prednison_equivs/more_than_70_dose_6.png
✓ Chart saved: out/cpzp/prednison_equivs/between_50_and_70_dose_1.png
✓ Chart saved: out/cpzp/prednison_equivs/between_50_and_70_dose_2.png
✓ Chart saved: out/cpzp/prednison_equivs/between_50_and_70_dose_3.png
✓ Chart saved: out/cpzp/prednison_equivs/between_5